# Classification using Deep Learning with Histogram data

---

### Reading the data

First, we’ll load the saved image and label data from the NumPy files.

In [11]:
import numpy as np
import gc

# Load training data
train_images = np.load('data/train_images.npy')
train_labels = np.load('data/train_labels.npy')
val_images = np.load('data/val_images.npy')
val_labels = np.load('data/val_labels.npy')


In [12]:
import numpy as np

# Carregar os arquivos
train_images = np.load('data/train_images.npy')
train_labels = np.load('data/train_labels.npy')
val_images = np.load('data/val_images.npy')
val_labels = np.load('data/val_labels.npy')

# Exibir informações das imagens
print("=== Características das Imagens ===")
print(f"Shape do conjunto de treino: {train_images.shape} (Amostras, Altura, Largura, Canais)")
print(f"Shape do conjunto de validação: {val_images.shape} (Amostras, Altura, Largura, Canais)")

# Resolução das imagens (altura e largura)
image_resolution = train_images.shape[1:3]  # Pegando as dimensões de altura e largura
print(f"Resolução das imagens: {image_resolution} pixels")

# Número total de imagens
print(f"Número de imagens no conjunto de treino: {train_images.shape[0]}")
print(f"Número de imagens no conjunto de validação: {val_images.shape[0]}")

# Tipo de dado armazenado
print(f"Tipo de dado das imagens: {train_images.dtype}")

# Intervalo de valores das imagens
print(f"Valores mínimo e máximo no dataset de treino: {train_images.min()} a {train_images.max()}")

# Exibir algumas amostras de rótulos únicos e suas contagens no conjunto de treino
unique_labels_train, counts_train = np.unique(train_labels, return_counts=True)
print("\n=== Distribuição das Classes no Conjunto de Treino ===")
for label, count in zip(unique_labels_train, counts_train):
    print(f"Classe {label}: {count} amostras")

# Exibir algumas amostras de rótulos únicos e suas contagens no conjunto de validação
unique_labels_val, counts_val = np.unique(val_labels, return_counts=True)
print("\n=== Distribuição das Classes no Conjunto de Validação ===")
for label, count in zip(unique_labels_val, counts_val):
    print(f"Classe {label}: {count} amostras")


=== Características das Imagens ===
Shape do conjunto de treino: (9711, 512, 512, 3) (Amostras, Altura, Largura, Canais)
Shape do conjunto de validação: (3237, 512, 512, 3) (Amostras, Altura, Largura, Canais)
Resolução das imagens: (512, 512) pixels
Número de imagens no conjunto de treino: 9711
Número de imagens no conjunto de validação: 3237
Tipo de dado das imagens: uint8
Valores mínimo e máximo no dataset de treino: 0 a 255

=== Distribuição das Classes no Conjunto de Treino ===
Classe 0: 2149 amostras
Classe 1: 635 amostras
Classe 2: 1186 amostras
Classe 3: 2140 amostras
Classe 4: 3601 amostras

=== Distribuição das Classes no Conjunto de Validação ===
Classe 0: 727 amostras
Classe 1: 222 amostras
Classe 2: 421 amostras
Classe 3: 721 amostras
Classe 4: 1146 amostras


## Juntando conjuntos de treino e validação

In [16]:
import numpy as np
import os

# Carregar os arquivos
train_images = np.load('data/train_images.npy')
train_labels = np.load('data/train_labels.npy')
val_images = np.load('data/val_images.npy')
val_labels = np.load('data/val_labels.npy')

# Criar diretório para salvar os dados somados
cached_data_dir = 'cached_data'
os.makedirs(cached_data_dir, exist_ok=True)

# Concatenar os conjuntos de treino e validação
all_images = np.concatenate((train_images, val_images), axis=0)
all_labels = np.concatenate((train_labels, val_labels), axis=0)

# Salvar os arquivos combinados
np.save(os.path.join(cached_data_dir, 'all_images.npy'), all_images)
np.save(os.path.join(cached_data_dir, 'all_labels.npy'), all_labels)

# Exibir informações das imagens
print("=== Características das Imagens ===")
print(f"Shape do conjunto combinado: {all_images.shape} (Amostras, Altura, Largura, Canais)")
print(f"Resolução das imagens: {all_images.shape[1:3]} pixels")
print(f"Número total de imagens: {all_images.shape[0]}")
print(f"Tipo de dado das imagens: {all_images.dtype}")
print(f"Valores mínimo e máximo no dataset combinado: {all_images.min()} a {all_images.max()}")

# Exibir distribuição das classes
unique_labels, counts = np.unique(all_labels, return_counts=True)
print("\n=== Distribuição das Classes no Conjunto Combinado ===")
for label, count in zip(unique_labels, counts):
    print(f"Classe {label}: {count} amostras")

print(f"\nDados salvos em: {cached_data_dir}")


=== Características das Imagens ===
Shape do conjunto combinado: (12948, 512, 512, 3) (Amostras, Altura, Largura, Canais)
Resolução das imagens: (512, 512) pixels
Número total de imagens: 12948
Tipo de dado das imagens: uint8
Valores mínimo e máximo no dataset combinado: 0 a 255

=== Distribuição das Classes no Conjunto Combinado ===
Classe 0: 2876 amostras
Classe 1: 857 amostras
Classe 2: 1607 amostras
Classe 3: 2861 amostras
Classe 4: 4747 amostras

Dados salvos em: cached_data


### Aplicando SMOTE para balancear classes

In [17]:
import numpy as np
import os
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle

# Diretórios
smote_dir = os.path.join(cached_data_dir, "SMOTE")
os.makedirs(smote_dir, exist_ok=True)

# Carregar os dados
all_images = np.load(os.path.join(cached_data_dir, "all_images.npy"))
all_labels = np.load(os.path.join(cached_data_dir, "all_labels.npy"))

# Reshape para formato 2D necessário para o SMOTE
num_samples, height, width, channels = all_images.shape
all_images_reshaped = all_images.reshape(num_samples, -1)  # Flatten imagens para (num_samples, features)

# Aplicar SMOTE para balancear as classes
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(all_images_reshaped, all_labels)

# Retornar ao formato original (altura, largura, canais)
X_resampled = X_resampled.reshape(-1, height, width, channels)

# Embaralhar os dados para evitar padrões sequenciais
X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

# Salvar os novos arquivos balanceados
np.save(os.path.join(smote_dir, "all_images_smote.npy"), X_resampled)
np.save(os.path.join(smote_dir, "all_labels_smote.npy"), y_resampled)

# Exibir distribuição das novas classes
unique_labels, counts = np.unique(y_resampled, return_counts=True)
print("\n=== Distribuição das Classes após SMOTE ===")
for label, count in zip(unique_labels, counts):
    print(f"Classe {label}: {count} amostras")

print(f"\nDados balanceados salvos em: {smote_dir}")



=== Distribuição das Classes após SMOTE ===
Classe 0: 4747 amostras
Classe 1: 4747 amostras
Classe 2: 4747 amostras
Classe 3: 4747 amostras
Classe 4: 4747 amostras

Dados balanceados salvos em: cached_data/SMOTE


### DIVIDINDO O CONJUNTO SOMADO E BALANCEANDO NA PROPORÇÃO 0.8 (TREINO)/ 0.2 (VALID)

In [18]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Diretórios de origem e destino
base_dir = "cached_data/SMOTE"
train_dir = "cached_data/SMOTE_TREINO"
valid_dir = "cached_data/SMOTE_VALID"

# Criar diretórios de saída
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

# Carregar os dados balanceados
all_images = np.load(os.path.join(base_dir, "all_images_smote.npy"))
all_labels = np.load(os.path.join(base_dir, "all_labels_smote.npy"))

# Separar os dados por classe e dividir em 80% treino, 20% validação
train_images_list, valid_images_list = [], []
train_labels_list, valid_labels_list = [], []

unique_labels = np.unique(all_labels)

for label in unique_labels:
    # Filtrar imagens e rótulos da classe atual
    indices = np.where(all_labels == label)[0]
    images_class = all_images[indices]
    labels_class = all_labels[indices]
    
    # Divisão em treino (80%) e validação (20%)
    train_images, valid_images, train_labels, valid_labels = train_test_split(
        images_class, labels_class, test_size=0.2, random_state=42, stratify=labels_class
    )
    
    # Armazenar os dados separados
    train_images_list.append(train_images)
    train_labels_list.append(train_labels)
    valid_images_list.append(valid_images)
    valid_labels_list.append(valid_labels)

# Concatenar os dados de todas as classes
train_images_final = np.concatenate(train_images_list, axis=0)
train_labels_final = np.concatenate(train_labels_list, axis=0)
valid_images_final = np.concatenate(valid_images_list, axis=0)
valid_labels_final = np.concatenate(valid_labels_list, axis=0)

# Salvar os arquivos divididos
np.save(os.path.join(train_dir, "train_images.npy"), train_images_final)
np.save(os.path.join(train_dir, "train_labels.npy"), train_labels_final)
np.save(os.path.join(valid_dir, "val_images.npy"), valid_images_final)
np.save(os.path.join(valid_dir, "val_labels.npy"), valid_labels_final)

# Exibir distribuição final das classes
unique_train, count_train = np.unique(train_labels_final, return_counts=True)
unique_valid, count_valid = np.unique(valid_labels_final, return_counts=True)

print("\n=== Distribuição das Classes no Conjunto de Treino ===")
for label, count in zip(unique_train, count_train):
    print(f"Classe {label}: {count} amostras")

print("\n=== Distribuição das Classes no Conjunto de Validação ===")
for label, count in zip(unique_valid, count_valid):
    print(f"Classe {label}: {count} amostras")

print(f"\nDados salvos em:\nTreino -> {train_dir}\nValidação -> {valid_dir}")



=== Distribuição das Classes no Conjunto de Treino ===
Classe 0: 3797 amostras
Classe 1: 3797 amostras
Classe 2: 3797 amostras
Classe 3: 3797 amostras
Classe 4: 3797 amostras

=== Distribuição das Classes no Conjunto de Validação ===
Classe 0: 950 amostras
Classe 1: 950 amostras
Classe 2: 950 amostras
Classe 3: 950 amostras
Classe 4: 950 amostras

Dados salvos em:
Treino -> cached_data/SMOTE_TREINO
Validação -> cached_data/SMOTE_VALID


In [21]:
import numpy as np
import gc

# Load training data
# train_images = np.load('data/train_images.npy')
# train_labels = np.load('data/train_labels.npy')
# val_images = np.load('data/val_images.npy')
# val_labels = np.load('data/val_labels.npy')

train_images = np.load('cached_data/SMOTE_TREINO/train_images.npy')
train_labels = np.load('cached_data/SMOTE_TREINO/train_labels.npy')
val_images = np.load('cached_data/SMOTE_VALID/val_images.npy')
val_labels = np.load('cached_data/SMOTE_VALID/val_labels.npy')

# Define quantile levels (e.g., deciles for 10 bins)
quantile_levels = np.linspace(0, 1, num=11)  # 0.0, 0.1, ..., 1.0
bin_edges = []

# Compute quantiles for each channel (R, G, B)
for channel in range(3):
    channel_pixels = train_images[:, :, :, channel].flatten()
    edges = np.quantile(channel_pixels, quantile_levels)
    bin_edges.append(edges)
# Save bin_edges for later use (e.g., validation)
np.save('cached_data/bin_edges.npy', bin_edges)

In [22]:
def image_to_histogram(image, bin_edges):
    features = []
    for channel in range(3):
        pixels = image[:, :, channel].flatten()
        hist, _ = np.histogram(pixels, bins=bin_edges[channel])
        hist = hist / len(pixels)  # Normalize to proportions
        features.extend(hist)
    return np.array(features)

# Convert training images to histograms
train_histograms = np.array([image_to_histogram(img, bin_edges) for img in train_images])
print("Created histograms from images")

# Free memory
del train_images
gc.collect()
print("train_images removed from memory.")

Created histograms from images
train_images removed from memory.


---

### Train CubeCatNet DNN model

We will define and train a Dense Neural Network (DNN) model.

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense



model = Sequential([
    Dense(128, activation='relu', input_shape=(train_histograms.shape[1],)),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train with early stopping
history = model.fit(
    train_histograms, to_categorical(train_labels, 5),
    epochs=30,
    batch_size=32,
    callbacks=[EarlyStopping(monitor='accuracy', patience=3)]
)

/home/omar/repos/CubeSat_ImageClassify/env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8282 - loss: 0.6168
Epoch 2/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9528 - loss: 0.1250
Epoch 3/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9605 - loss: 0.1063
Epoch 4/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 969us/step - accuracy: 0.9673 - loss: 0.0923
Epoch 5/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9711 - loss: 0.0799
Epoch 6/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 954us/step - accuracy: 0.9738 - loss: 0.0718
Epoch 7/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 984us/step - accuracy: 0.9769 - loss: 0.0679
Epoch 8/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 955us/step - accuracy: 0.9788 - loss: 0.0614
Epoch 9/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9790 - loss: 0.0590
Epoch 10/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 977us/step - accuracy: 0.9806 - loss: 0.0567
Epoch 11/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9806 - loss: 0.0523
Epoch 12/30
594/594 ━━━━━━━━━━━━━━━━━━━━ 1

##### **Saving the DNN model**

In [24]:
import pickle

with open('dnn_histogram_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [25]:
import gc

del train_labels
gc.collect()
print("train_labels removed from memory.")

train_labels removed from memory.


---

### Deep learning: Validation set results

In [26]:
val_images = np.load('data/val_images.npy')  # Load image val data
val_labels = np.load('data/val_labels.npy')  # Load label val data
val_labels = to_categorical(val_labels, num_classes=5)

In [27]:
with open('dnn_histogram_model.pkl', 'rb') as file:
    cnn_loaded_model = pickle.load(file)
    
# Load precomputed bin edges
bin_edges = np.load('cached_data/bin_edges.npy')

# Convert validation images to histograms
val_images = np.load('data/val_images.npy')
val_histograms = np.array([image_to_histogram(img, bin_edges) for img in val_images])

# Free memory
del val_images
gc.collect()

# Now use histograms for prediction
val_predictions = cnn_loaded_model.predict(val_histograms)

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step


In [35]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert from one-hot or probability distributions to single integer class indices (not necessary for 1D array)
#val_predictions = np.argmax(val_predictions, axis=1) 
#val_labels = np.argmax(val_labels, axis=1) 



# Detailed classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_predictions))


Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       727
           1       1.00      1.00      1.00       222
           2       1.00      1.00      1.00       421
           3       0.98      0.99      0.99       721
           4       1.00      1.00      1.00      1146

    accuracy                           0.99      3237
   macro avg       0.99      0.99      0.99      3237
weighted avg       0.99      0.99      0.99      3237



In [36]:
import plotly.figure_factory as ff
import numpy as np
from sklearn.metrics import confusion_matrix

# Define class names
class_names = ["Blurry", "Corrupt", "Missing_Data", "Noisy", "Priority"]

# Compute the confusion matrix
cm = confusion_matrix(val_labels, val_predictions)

# Create the heatmap
fig = ff.create_annotated_heatmap(
    z=cm, 
    x=class_names, 
    y=class_names, 
    colorscale="Blues",
    showscale=True
)

# Update layout for better readability
fig.update_layout(
    title="Confusion Matrix with Class Names",
    xaxis=dict(title="Predicted Label"),
    yaxis=dict(title="True Label")
)

# Show the figure
fig.show()


---

# Deep learning: testing with unseen data

In [37]:
import numpy as np
from keras.utils import to_categorical
from sklearn.metrics import classification_report
import pickle
import gc

def test_models():
    # Load test data
    test_images = np.load('data/test_images1.npy')
    test_labels = np.load('data/test_labels.npy')
    true_classes = test_labels  # Save original labels for reporting

    # Test Histogram Model
    print("\nTesting Histogram-Based Model:")
    
    # Load histogram model and bin edges
    with open('dnn_histogram_model.pkl', 'rb') as f:  # Assuming you saved this separately
        hist_model = pickle.load(f)
    
    bin_edges = np.load('cached_data/bin_edges.npy')
    
    # Convert test images to histograms
    test_histograms = np.array([image_to_histogram(img, bin_edges) for img in test_images])
    
    # Predict with histograms
    hist_pred_probs = hist_model.predict(test_histograms)
    hist_pred_classes = np.argmax(hist_pred_probs, axis=1)
    
    # Generate report
    print("Histogram Model Report:")
    print(classification_report(true_classes, hist_pred_classes))
    
    # Cleanup
    del test_images, test_histograms, hist_model
    gc.collect()

def image_to_histogram(image, bin_edges):
    """ Your existing histogram conversion function """
    features = []
    for channel in range(3):
        pixels = image[:, :, channel].flatten()
        hist, _ = np.histogram(pixels, bins=bin_edges[channel])
        hist = hist / len(pixels)  # Normalize
        features.extend(hist)
    return np.array(features)

test_models()


Testing Histogram-Based Model:
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
Histogram Model Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       668
           1       1.00      1.00      1.00       213
           2       1.00      1.00      1.00       414
           3       0.99      0.98      0.99       721
           4       1.00      1.00      1.00      1221

    accuracy                           0.99      3237
   macro avg       0.99      0.99      0.99      3237
weighted avg       0.99      0.99      0.99      3237



---